# Exploratory Data Analysis and Data Cleaning <a name="Top"></a>

# TABLE OF CONTENT

[ABOUT THIS DATASET](#About)  
1. [IMPORT LIBRARIES](#ImportLibraries)  
2. [DATA EXPLORATION AND DATA CLEANING](#DataExplDataClean)  
    2.1 [Data types](#DataTypes_df1)      
    2.2 [Missing values](#Missing_df1)       
    2.3 [Uniform Values](#UniformValues_df1)     
    2.4 [Remove Duplicates](#RemDupl_df1)    
    2.5 [Numeric data](#NumData_df1)    
    2.6 [Categorical data](#CatData_df1)    
3. [DATA ANALYSIS](#DataAnalysis) 
4. [FINAL CONCLUSIONS](#Conclusions) <a name="Conclusions"></a>

# ABOUT THIS DATASET <a name="About"></a>

This dataset provides detailed insights into Amazon sales data, including SKU Code, Design Number, Stock, Category and Size, to help optimize product profitability. It covers a 90 days time frame, from 31 March 2022 to 29 June 2022.

**Attributes:**
* **Order ID**: Order Identification Number. (String)
* **Date**: Date of the sale. (Date)
* **Status**: Status of the sale. (String)
* **Fulfilment**: Method of fulfilment. (String)
* **Sales Channel**: Method of selling. (String)
* **Ship Service Level**:  Method of Shipping Service. (String)  
* **Style**: Style of the product. (String)
* **SKU**: Stock Keeping Unit. (String)
* **Category**: Type of product. (String)
* **Size**: Size of the product. (String)
* **ASIN**: Amazon Standard Identification Number. (String)
* **Courier Status**: Status of the courier. (String)
* **Qty**: Quantity of the product. (Integer)
* **Currency**: The currency used for the sale (INR). (String)
* **Amount**: Amount of the sale. (Float)
* **Ship City**: Destination City of the shipment. (String)
* **Ship State**: Destination State of the shipment. (String)
* **Ship Postal Code**: Destination Postal Code of the shipment. (String)
* **Ship Country**: Destination Country of the shipment. (String)
* **Promotion IDS**: Promotion code of the sale. (String)
* **B2B**: Business to business sale. (Boolean)

 
**Questions to be answered:**

1. Which categories have sold most?  [[&#8599;]](#AQ1)    
2. Which are the 20 Best-Selling Products?   [[&#8599;]](#AQ2)    
3. Which are the 20 Best-Selling Products by Revenue?   [[&#8599;]](#AQ3)    
4. Which are the 20 cities that made the most orders?   [[&#8599;]](#AQ4)    
5. Which are the 20 cities that more income generate?    [[&#8599;]](#AQ5)    
6. How many orders are fulfilled by Amazon and how many by the Merchant?    [[&#8599;]](#AQ6)    

# 1. IMPORT LIBRARIES [&#129081;](#Top)<a name="ImportLibraries"></a>

**LIBRARIES:**

* Library **pandas**: to handle data in tabular representation.
* Library **missingno**: to graphically display missing values in the data.
* Library **re**: to work with regular expressions.
* Library **matplotlib, seaborn**: to visualize data in in plots, graphs and charts.

In [ ]:
import pandas as pd
import missingno as msno
import re
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
import seaborn as sns

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

# 2 DATA DESCRIPTION AND DATA CLEANING [&#129081;](#Top)<a name="DataExplDataClean"></a>
In this phase, will be performed the exploratory data analysis: data types, missing values, string standardisation, duplicates, measures of the central trend, and relationship detection between fields.

## 2.1 Data types [&#129081;](#Top)<a name="DataTypes_df1"></a>

In [ ]:
# Load data
df = pd.read_csv("Data in\\Amazon Sale Report.csv", low_memory=False) 

In [ ]:
df.head() # Display the first 5 lines of the dataset

* Check the dimension of the data

In [ ]:
df_sizes = {'start': df.shape}

In [ ]:
print(f"Raw data contains {df_sizes['start'][0]} records and {df_sizes['start'][1]} columns.")

* Standardize the column names

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={'Order ID': 'Order_ID', 'Sales Channel ': 'Sales_channel',
        'ship-service-level': 'ship_service_level', 'ship-city': 'ship_city', 
        'ship-state': 'ship_state', 'ship-postal-code': 'ship_postal_code', 'ship-country': 'ship_country', 
        'promotion-ids': 'promotion_ids','Courier Status': 'Courier_Status', 'currency': 'Currency', 
        'fulfilled-by':'fulfilled_by'})

* Check columns data type

In [ ]:
df.info()

The dataset contains 24 columns, of which 4 containig numeric values:
* index  (int)
* Qty (int)
* Amount (float)
* ship-postal-code (float), later it will be converted to integer

* Set the column 'index' as the index of the dataframe. With this operation the number of columns becomes 22.

In [ ]:
df.set_index('index', inplace = True)

## 2.2 Missing values [&#129081;](#Top)<a name="Missing_df1"></a>
### Evaluate how many values are missing for each attribute and how they are distributed.

* Create a separate dataframe containing the informations about missing and unique values for each column, to better see the correlations.

In [ ]:
def missing_evaluation(dataframe):
    '''
    missing_evaluation(dataframe)
    
    Returns a dataframe containing the informations about missing and unique values 
    for each column on a dataframe passed as a parameter

    Parameters: 
    dataframe: dataframe to analyze
    '''
    
    # Column 'qt_missing': Number of NaN values
    df_missing_stats = pd.DataFrame(data=dataframe.isna().sum(), index=dataframe.columns, columns=['qt_missing'])
    # Column 'nr_unique_values': Number of unique values
    df_missing_stats['qt_unique_values'] = pd.DataFrame(data=dataframe.nunique(), index=dataframe.columns)
    # Column 'unique_values': unique values of the attributes
    df_missing_stats['unique_values'] = pd.DataFrame(data=dataframe.apply(pd.unique), index=dataframe.columns)
    # Column 'perc_missing': percentage of missing values
    df_missing_stats['perc_missing'] = pd.DataFrame(data=dataframe.isnull().mean())
    return df_missing_stats

In [ ]:
df_missing_stats = missing_evaluation(df)
df_missing_stats

In [ ]:
# Visualize the distribution of missing values with missingno matrix
msno.matrix(df);

In [ ]:
# Use the heatmap to identify correlations of the nullity between each of the different columns.
msno.heatmap(df);

Worth noting that:
* When one of the columns **'Amount'** and **'Currency'** is null, the other is also null (7795 raws with null values).
* When one of columns **'ship_city', 'ship_state', 'ship_postal_code', 'ship_country'** has a null value, the others are also null (33 raws with null values).

### Fill in (or eliminate) the missing values with a case-by-case assessment.
After the exploration of the missing values, it's possible to proceed fill (or clean) each column, performing an individual assessment.

* **Courier_Status**: missing values filled with 'Unknown'

In [ ]:
# Replace NaN values in column 'Courier_Status' with the value 'Unknown'
df['Courier_Status'].fillna('Unknown', inplace = True)

* Check nullity correlation on **Amount** and **Currency** columns
* Fill missing values in **Amount** with zero
* Fill missing values in **Currency** with 'INR'

In [ ]:
set_idx_amount_null = set(df[df['Amount'].isnull()].index)

In [ ]:
set2_idx_currency_null = set(df[df['Currency'].isnull()].index)

In [ ]:
print(f"Question: Columns 'Amount' and 'Currency' have 1 to 1 nullity correlation?\nAnswer: {set_idx_amount_null == set2_idx_currency_null}") 

In [ ]:
df['Amount'].fillna(0, inplace = True)

In [ ]:
df['Currency'].fillna('INR', inplace = True)

* **promotion_ids**: missing values filled with 'No Promo'

In [ ]:
# Replace NaN values in column 'promotion_ids' with the value 'Unknown'
df['promotion_ids'].fillna('No Promo', inplace = True)

* **ship_city** and **ship_state**: missing values filled with 'Unknown'
* **ship_postal_code**: missing values filled with zero
* **ship_country**: missing values filled with 'Unknown'

In [ ]:
df['ship_city'].fillna('Unknown', inplace = True)
df['ship_state'].fillna('Unknown', inplace = True)
df['ship_postal_code'].fillna(0, inplace = True)
df['ship_country'].fillna('Unknown', inplace = True)

* **fulfilled_by**: missing values filled with 'FBA' (Fulfilled by Amazon).

In [ ]:
df['fulfilled_by'].fillna('FBA', inplace = True)

* **Unnamed: 22** will be deleted because it isn't relevant

In [ ]:
df.drop(columns = ['Unnamed: 22'], inplace = True)

### Check that none of the missing values are left

In [ ]:
df.isna().sum().sum()

In [ ]:
df_missing_stats = missing_evaluation(df)
df_missing_stats

## 2.3 Uniform the values in each column [&#129081;](#Top)<a name="UniformValues_df1"></a>
To make values consistent, such as eliminating duplicates and having unique values, it is first of all necessary to standardize the style of the strings.
It's a procedure in which the intervention to apply for each column must be evaluated individually.

* **Order_ID**: it's proper that it's a string, remove extra white spaces from the beginning and end, and set the characters in UPPER case.

In [ ]:
df['Order_ID'] = df['Order_ID'].str.strip()

In [ ]:
df['Order_ID'] = df['Order_ID'].str.upper()

* **Date**: it's in string format, set it to to dtype datetime. 

In [ ]:
# Set column 'Date' to data type datetime
df['Date'] = pd.to_datetime(df['Date'])

* **Status**: it's proper that it's a string, remove extra white spaces from the beginning and end.   
Then check if there are the same 'Status' values but with different spellings.  
As verified above, it contains 13 values, so they can be checked visually one by one, avoiding apply without distinction Capitalize or UPPER methods, keeping the contents legible at a glance within possible limits.

In [ ]:
df['Status'] = df['Status'].str.strip()

In [ ]:
# Check column 'Status', result: it doesn't need intervention
for value in df.Status.sort_values().unique():
    print(f"'{value}'")

* **Fulfilment**: contains 2 unique values ('Merchant' and 'Amazon'), and are already written in the correct form
* **Sales_channel**: contains 2 unique values ('Amazon.in' and 'Non-Amazon'), and are already written in the correct form
* **ship_service_level**: contains 2 unique values('Standard' and 'Expedited'), and are already written in the correct form

* **Style** and **SKU**: it's proper that they are strings, remove extra white spaces from the beginning and end, and set the characters in UPPER case.

In [ ]:
df['Style'] = df['Style'].str.strip()
df['Style'] = df['Style'].str.upper()
df['SKU'] = df['SKU'].str.strip()
df['SKU'] = df['SKU'].str.upper()

* **Category**: it's proper that it's a string, remove extra white spaces from the beginning and end, and set the characters in UPPER case.

In [ ]:
df['Category'] = df['Category'].str.strip()
df['Category'] = df['Category'].str.upper()

* **Size**: contains 11 unique values, and as can be seen from the table above they are already written in the correct form.

* **ASIN**: it's proper that it's a string, remove extra white spaces from the beginning and end, and set the characters in UPPER case.

In [ ]:
df['ASIN'] = df['ASIN'].str.strip()
df['ASIN'] = df['ASIN'].str.upper()

* **Courier_Status**: contains 4 unique values (Shipped, Cancelled, Unshipped, Unknown), and are already written in the correct form.   
* **Qty**: is a integer number, with max value of 15 units. It doesn't nedd intervention. 
* **Currency**: contains 1 unique value (INR), and is already written in the correct form.  
* **Amount**: is a float number. With the measurements made so far it doesn't seem to need intervention.

* **ship_city** and **ship_state**: it’s proper that they are strings, remove extra white spaces from the beginning and end, and uniform all the characters to UPPER case. Then check the accuracy of geographic names. And normalize different istances of the same state and city.

In [ ]:
df['ship_city'] = df['ship_city'].str.strip()
df['ship_city'] = df['ship_city'].str.upper()
df['ship_state'] = df['ship_state'].str.strip()
df['ship_state'] = df['ship_state'].str.upper()

In [ ]:
df.loc[df['ship_state'].isin(['AR']),'ship_state'] = 'ARUNACHAL PRADESH'
df.loc[df['ship_state'].isin(['NEW DELHI']),'ship_state'] = 'DELHI'
df.loc[df['ship_state'].isin(['NL']),'ship_state'] = 'NAGALAND'
df.loc[df['ship_state'].isin(['ORISSA']),'ship_state'] = 'ODISHA'
df.loc[df['ship_state'].isin(['PONDICHERRY']),'ship_state'] = 'PUDUCHERRY'
df.loc[df['ship_state'].isin(['PUNJAB/MOHALI/ZIRAKPUR','PB']),'ship_state'] = 'PUNJAB'
df.loc[df['ship_state'].isin(['RAJSHTHAN','RAJSTHAN','RJ']),'ship_state'] = 'RAJASTHAN'

In [ ]:
df['ship_city'] = df['ship_city'].replace(['PUDUCHERRY','PONDYCHERRY', 'PUDUCHERRY 605001', 'PUDUCHERRY 605003'],'PUDUCHERRY')

In [ ]:
df['ship_city'] = df['ship_city'].replace(['NORTH DELHI', 'NORTH WEST DELHI','NEW DELHI (NORTH)'],'NEW DELHI (NORTH)')
df['ship_city'] = df['ship_city'].replace(['SOUTH  DELHI', 'SOUTH DELHI'],'NEW DELHI (SOUTH)')
df['ship_city'] = df['ship_city'].replace(['SOUTH EAST DELHI'],'NEW DELHI (SOUTH EAST)')
df['ship_city'] = df['ship_city'].replace(['SOUTH WEST DELHI','SOUTH-WEST DELHI'],'NEW DELHI (SOUTH WEST)')
df['ship_city'] = df['ship_city'].replace(['WEST DELHI'],'NEW DELHI (WEST)')
df['ship_city'] = df['ship_city'].replace(['CENTRAL DELHI','CITY'],'NEW DELHI (CENTRAL)')
df['ship_city'] = df['ship_city'].replace(['EAST DELHI'],'NEW DELHI (EAST)')

In [ ]:
pattern = r'^NEW DELHI(?!.*\()'
df['ship_city'] = df['ship_city'].apply(lambda x: "NEW DELHI" if re.match(pattern, x) else x)

In [ ]:
df['ship_city'] = df['ship_city'].replace(['N.DELHI','NEW DELH','DELHI','DELHI -86','DELHI 110085','DELHI CANTT','DELHI-92.','DELHIQ', 'NEW-DELHI',
 'NEWDELHI'],'NEW DELHI')

In [ ]:
pattern = r'^BANGALOR(?!.*\()'
df['ship_city'] = df['ship_city'].apply(lambda x: 'BANGALORE' if re.match(pattern, x) else x)
df['ship_city'] = df['ship_city'].apply(lambda x: 'BANGALORE' if re.match(r'^BENGALUR(?!.*\()', x) else x)
df['ship_city'] = df['ship_city'].replace(['BENGALOORU','BENGOLOOR'],'BANGALORE')

Note: since this notebook is for demonstration purposes only, I normalized all State names, and only the cities of Puducherry, New Delhi and Bangalore.

* **ship_postal_code**: it's currently a float, but postal code is a categorical data, so it needs to be converted in string. Remember that all missing values have been replaced with zeros.

In [ ]:
df['ship_postal_code'] = df['ship_postal_code'].astype(str)

In [ ]:
df.info()

* **promotion_ids**: it's proper that it is a string, remove extra white spaces from the beginning and end, and set the characters in UPPER case.

In [ ]:
df['promotion_ids'] = df['promotion_ids'].str.strip()
df['promotion_ids'] = df['promotion_ids'].str.upper()

* **B2B**: boolean, True when the sale is Business to Business, False when is Business to Consumer. It doesn't needs intervention.
* **fulfilled_by**: contains 2 unique values (Easy Ship, FBA). It doesn't needs intervention.

## 2.4 Remove Duplicates [&#129081;](#Top)<a name="RemDupl_df1"></a>

* Remove duplicate raws

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df_sizes.update({'after_remove_duplicates': df.shape})
print(f"Deleted {df_sizes['start'][0] - df_sizes['after_remove_duplicates'][0] } raws, {df_sizes['after_remove_duplicates'][0]} remain.")

* Remove **fulfilled_by** column, bacause it doesn't add information to data. In fact it contains the same information as the column **Fulfilment**.

In [ ]:
print(f"""
Question: When the value in column 'Fulfilment' is 'Merchant', the value in column 'fulfilled_by' is 'Easy Ship'?
Answer:   {set(df.query("Fulfilment == 'Merchant'").index) == set(df.query("fulfilled_by == 'Easy Ship'").index)}""") 
print(f"""
Question: When the value in column 'Fulfilment' is 'Amazon', the value in column 'fulfilled_by' is 'FBA'?
Answer:   {set(df.query("Fulfilment == 'Merchant'").index) == set(df.query("fulfilled_by == 'Easy Ship'").index)}""") 

In [ ]:
df.drop(columns = ['fulfilled_by'], inplace = True)

## 2.5 Numeric data [&#129081;](#Top)<a name="NumData"></a>

In [ ]:
df.describe(datetime_is_numeric=True) # only numerical Attributes (including Date)

In [ ]:
df['Date'].max() - df['Date'].min()

The start date of the dataset is 31 March 2022, the end date is 29 June 2022, over a 90 day period.

## 2.6 Categorical data [&#129081;](#Top)<a name="CatData"></a>

In [ ]:
df.describe(include='O').T # only numpy.object data type Categorical Data

* Add the column **Month**

In [ ]:
df['month'] = df['Date'].dt.month
df['month'].unique()

In [ ]:
df['month'].replace([3,4,5,6],['March','April', 'May', 'June'], inplace = True)

* Save the cleaned dataframe in a csv file

In [ ]:
df.to_csv("Data out\\new_df_Amazon_sales.csv")

# 3. DATA ANALYSIS <a name="DataAnalysis"></a>

## Answer to Q1: Which categories have sold most? [&#129081;](#Top)<a name="AQ1"></a>

In [ ]:
# Which categories have sold most?
df_q1 = df[['Amount', 'Qty']].groupby(df['Category']).sum()\
                                .sort_values('Amount', ascending=False)\
                                .reset_index()
df_q1

In [ ]:
# define font sizes
_title_size = 18
_axis_label_size = 12
_label_size = 10
_axis_label_pad = 10
_legend_title_size = 12
_legend_content_size = 12
_padding_title = 20
_fweight = 'normal'

In [ ]:
plt.figure(figsize = (12, 6))
ax_q1 = sns.barplot(y = 'Amount', x='Category', palette = ['#3182bd'], data = df_q1)
plt.title('Revenue by Categories', fontsize= _title_size)

formatter = ticker.StrMethodFormatter("{x:,.0f}.")
ax_q1.yaxis.set_major_formatter(formatter)
ax_q1.xaxis.labelpad = _axis_label_pad
ax_q1.yaxis.labelpad = _axis_label_pad

for i, bar in enumerate(ax_q1.containers[0].patches):
    ax_q1.text(bar.get_x() + bar.get_width()/2., bar.get_height(),
        '{:,.0f}'.format(bar.get_height()), ha='center', va='bottom')
plt.xlabel("Category", weight = _fweight, fontsize=_axis_label_size)
plt.ylabel("Sales", weight = _fweight, fontsize=_axis_label_size)
plt.savefig('img\\Revenue_by_categories.jpg', dpi=300)
plt.show()


## Answer Q2: Which are the 20 Best-Selling Products? [&#129081;](#Top)<a name="AQ2"></a>

In [ ]:
df_q2 = df.groupby(['Category', 'ASIN'])[['Qty','Amount']].sum()\
                                     .sort_values('Qty', ascending=False)\
                                     .head(20)\
                                     .reset_index()
df_q2

In [ ]:
colors_dict = {'WESTERN DRESS': '#4BD9EC', 'KURTA': '#758BFD', 'SET': '#313E5B'} 
col_list = [colors_dict.get(cat) for cat in df_q2['Category']]

plt.figure(figsize = (12, 6))
ax_q2 = sns.barplot(y = 'Qty', x='ASIN', palette = col_list, data = df_q2)
plt.title('20 Best-Selling Products by Quantity', fontsize=_title_size)

formatter = ticker.StrMethodFormatter("{x:,.0f}")
ax_q2.yaxis.set_major_formatter(formatter)
ax_q2.xaxis.labelpad = _axis_label_pad
ax_q2.yaxis.labelpad = _axis_label_pad
ax_q2.margins(0.01, 0.2)

for i, bar in enumerate(ax_q2.containers[0].patches):
    ax_q2.text(bar.get_x() + bar.get_width()/2., bar.get_height(),
        '{:,.0f}'.format(bar.get_height()), ha='center', va='bottom')
plt.xticks(rotation=90, ha='center')
plt.xlabel("Product", weight = _fweight, fontsize= _axis_label_size)
plt.ylabel("Qty", weight = _fweight, fontsize= _axis_label_size)

legend_entries = []
for elem in df_q2['Category'].unique():
    legend_entries.append(mpatches.Patch(color=colors_dict.get(elem), label=elem))
plt.legend(handles=legend_entries, title='Category:', title_fontsize= _legend_title_size, fontsize=_legend_content_size)
plt.savefig('img\\20_Best_Selling_Products_by_Quantity.jpg', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()

## Answer to Q3: Which are the 20 Best-Selling Products by Revenue? [&#129081;](#Top)<a name="AQ3"></a>

In [ ]:
df_q3 = df.groupby(['Category', 'ASIN'])[['Amount', 'Qty']].sum()\
                                     .sort_values('Amount', ascending=False)\
                                     .head(20)\
                                     .reset_index()
df_q3

In [ ]:
colors_dict = {'WESTERN DRESS': '#4BD9EC', 'KURTA': '#758BFD', 'SET': '#313E5B'} 
col_list = [colors_dict.get(cat) for cat in df_q3['Category']]

plt.figure(figsize = (12, 6))
ax_q3 = sns.barplot(y = 'Amount', x='ASIN', palette = col_list, data = df_q3)
plt.title('20 Best-Selling Products by Revenue', fontsize=_title_size)

formatter = ticker.StrMethodFormatter("{x:,.0f}")
ax_q3.yaxis.set_major_formatter(formatter)
ax_q3.xaxis.labelpad = _axis_label_pad
ax_q3.yaxis.labelpad = _axis_label_pad

for i, bar in enumerate(ax_q3.containers[0].patches):
    ax_q3.text(bar.get_x() + bar.get_width()*0.8, bar.get_height(),
        '{:,.0f}'.format(bar.get_height()), ha='center', va='bottom', rotation=45)
    
plt.xticks(rotation=90, ha='center')
plt.xlabel("Product", weight = _fweight, fontsize= _axis_label_size)
plt.ylabel("Sales", weight = _fweight, fontsize= _axis_label_size)

legend_entries = []
for elem in df_q3['Category'].unique():
    legend_entries.append(mpatches.Patch(color=colors_dict.get(elem), label=elem))
plt.legend(handles=legend_entries, title='Category:', title_fontsize= _legend_title_size, fontsize=_legend_content_size)
ax_q3.margins(0.01, 0.2)
plt.savefig('img\\20_Best_Selling_Products_by_Revenue.jpg', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()

## Answer Q4: Which are the 20 cities that made the most orders? [&#129081;](#Top)<a name="AQ4"></a>

In [ ]:
df_q4 = df.groupby(['ship_state','ship_city'])[['Order_ID']].count()\
                                    .sort_values('Order_ID', ascending=False)\
                                    .head(20)\
                                    .reset_index()
df_q4

In [ ]:
plt.figure(figsize = (12, 6))

plt.title('First 20 Cities by Nr. of Orders', fontsize=_title_size)

palette_colors = sns.color_palette('Set2',len(df_q4['ship_state'].unique()))
palette_dict = {state: color for state, color in zip(list(df_q4['ship_state'].unique()), palette_colors)}
col_list = [palette_dict.get(state) for state, city in zip(df_q4['ship_state'],df_q4['ship_city'])]

ax_q4 = sns.barplot(y = 'Order_ID', x='ship_city', palette = col_list, data = df_q4)

formatter = ticker.StrMethodFormatter("{x:,.0f}")
ax_q4.yaxis.set_major_formatter(formatter)
ax_q4.yaxis.labelpad = _axis_label_pad

for i, bar in enumerate(ax_q4.containers[0].patches):
    ax_q4.text(bar.get_x() + bar.get_width()*0.7, bar.get_height(),
        '{:,.0f}'.format(bar.get_height()), ha='center', va='bottom', rotation=45)
plt.xticks(rotation=90)
plt.xlabel("City", weight = _fweight, fontsize= _axis_label_size)
plt.ylabel("Nr. of Orders", weight = _fweight, fontsize= _axis_label_size)

legend_entries = []
for elem in df_q4['ship_state'].unique():
    legend_entries.append(mpatches.Patch(color=palette_dict.get(elem), label=elem))
plt.legend(handles=legend_entries, title='State:', title_fontsize= _legend_title_size-1, fontsize=_legend_content_size-1)

ax_q4.margins(0.01, 0.2)
plt.savefig('img\\First_20_Cities_by_Nr_of_Orders.jpg', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()

## Answer Q5: Which are the 20 cities that more income generate? [&#129081;](#Top)<a name="AQ5"></a>

In [ ]:
df_q5 = df.groupby(['ship_state','ship_city'])[['Amount','Qty']].sum()\
                                    .sort_values('Amount', ascending=False)\
                                     .head(20)\
                                    .reset_index()

In [ ]:
plt.figure(figsize = (12, 6))
plt.title('First 20 Cities by Revenue', fontsize=_title_size)

palette_colors = sns.color_palette('Set2',len(df_q5['ship_state'].unique()))
palette_dict = {state: color for state, color in zip(list(df_q5['ship_state'].unique()), palette_colors)}
col_list = [palette_dict.get(state) for state, city in zip(df_q5['ship_state'],df_q5['ship_city'])]

ax_q5 = sns.barplot(y = 'Amount', x='ship_city', palette = col_list, data = df_q5)

formatter = ticker.StrMethodFormatter("{x:,.0f}")
ax_q5.yaxis.set_major_formatter(formatter)

ax_q5.yaxis.labelpad = _axis_label_pad

for i, bar in enumerate(ax_q5.containers[0].patches):
    ax_q5.text(bar.get_x() + bar.get_width()*0.7, bar.get_height(),
        '{:,.0f}'.format(bar.get_height()), ha='center', va='bottom', rotation=45)
plt.xticks(rotation=90)
plt.xlabel("City", weight = _fweight, fontsize= _axis_label_size)
plt.ylabel("Amount", weight = _fweight, fontsize= _axis_label_size)

legend_entries = []
for elem in df_q5['ship_state'].unique():
    legend_entries.append(mpatches.Patch(color=palette_dict.get(elem), label=elem))
plt.legend(handles=legend_entries, title='State:', title_fontsize= _legend_title_size, fontsize=_legend_content_size-1)

ax_q5.margins(0.02, 0.2)
plt.savefig('img\\First_20_Cities_by_Revenue.jpg', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()

## Answer Q6: How many orders are fulfilled by Amazon and how many by the Merchant? [&#129081;](#Top)<a name="AQ6"></a>

In [ ]:
# How are the order volume and revenues distributed on the Fulfilment Method?
df_fulfilment_stats = pd.DataFrame(data= df.groupby(['Fulfilment'])['Order_ID'].count())
df_fulfilment_stats = df_fulfilment_stats.rename(columns={'Order_ID': 'Nr. of Orders'})
df_fulfilment_stats['% of Orders'] = [ x / (df_fulfilment_stats['Nr. of Orders'].sum()) for x in df_fulfilment_stats['Nr. of Orders'] ]
df_fulfilment_stats['Tot. Sales'] = df.groupby(['Fulfilment'])[['Amount']].sum()
df_fulfilment_stats['% of Tot. Sales'] = [ x / (df_fulfilment_stats['Tot. Sales'].sum()) for x in df_fulfilment_stats['Tot. Sales'] ]
df_fulfilment_stats

In [ ]:
plt.figure(figsize = (8, 6))
fig_x2 = df['Fulfilment'].value_counts().plot.pie(autopct="%1.1f%%", colors = ('#FF9900','#146EB4'), 
                                                  textprops={'fontsize': 14})
fig_x2.set(ylabel=None)
plt.legend(title = 'Fulfilment', title_fontsize= _legend_title_size, fontsize=_legend_content_size, bbox_to_anchor=(1,0.8),
                          bbox_transform=plt.gcf().transFigure)
plt.title('Fulfilment Orders Overview', fontsize= _title_size);
plt.savefig('img\\Fulfilment_Orders_Overview.jpg', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
df_shipment_stats = df.query("Courier_Status != 'Unknown'").groupby(['Courier_Status','Fulfilment'])[['Amount','Qty']].sum()\
                                    .sort_values('Amount', ascending=False)\
                                    .reset_index()

In [ ]:
df_shipment_stats

In [ ]:
df_shipment_stats['Nr. of Orders'] = df.groupby(['Courier_Status','Fulfilment'])[['Order_ID']].count()\
                                    .sort_values('Order_ID', ascending=False)\
                                    .reset_index()['Order_ID']

In [ ]:
df_shipment_stats

In [ ]:
plt.figure(figsize = (12, 6))

ax6 = sns.countplot(x = 'Courier_Status', hue = 'Fulfilment', palette = ['#146EB4','#FF9900'], data = df.query("Courier_Status != 'Unknown'"));
plt.title('Shipping State Overview', fontsize=_title_size);
ax6.xaxis.labelpad = _axis_label_pad
ax6.yaxis.labelpad = _axis_label_pad

for i in ax6.containers:
    ax6.bar_label(i,)
plt.xlabel("Courier Status", weight = _fweight, fontsize= _axis_label_size)
plt.ylabel("Nr. of Orders", weight = _fweight, fontsize= _axis_label_size)
plt.legend(title='Fulfilment:', title_fontsize= _legend_title_size, fontsize=_legend_content_size)
plt.savefig('img\\Shipping_State_Overview.jpg', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show;

## FINAL CONCLUSIONS [&#129081;](#Top)<a name="Conclusions"></a>

In the course of the analysis:
* It was established that Western Dress, Kurta and Set are the categories which have sold most.
* It was verified that the 20 Best-Selling Products by quantity belongs to the categories Western Dress, Kurta and Set.  
* On the other hand, the 20 Best-Selling Products by revenue belongs only to the categories Western Dress and Set.  
* Also it was performed a fulfilment and shipping state overview.

   
---   
   




### Data Source
The data for this project can be found at [data.world](https://data.world/anilsharma87), thanks to [ANil](https://data.world/anilsharma87).

### Author 
[Barbara Callegari](https://numberslab.net)  
To learn more about the author visit her [LinkedIn Profile](https://www.linkedin.com/in/barbaracallegari)

### Licence
All rights reserved 2023. All code is **created and owned by Barbara Callegari**.   
**If you use his code, please give her a skill endorsement in Python and Data Analysis on LinkedIn**.   
Visit her at https://www.linkedin.com/in/barbaracallegari